In [2]:
import pandas as pd
import datetime as dt
import dask.dataframe as dd
import dateparser
import swifter
from fastparquet import ParquetFile
from datetime import timedelta
import time
import datetime
from multiprocessing import Pool, cpu_count
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load Chattanooga 2019 Data

These files are located in Teams in general > datasets > carta-occupancy > carta > Carta 2019 APC RideCheck Data > Carta 2019 APC RideCheck Data

In [11]:
jul_df = pd.read_csv('JUL2019_RIDECHECK_DATA.txt', index_col=0)
aug_df = pd.read_csv('AUG2019_RIDECHECK_DATA.txt', index_col=0)
sep_df = pd.read_csv('SEP2019_RIDECHECK_DATA.txt', index_col=0)
oct_df = pd.read_csv('OCT2019_RIDECHECK_DATA.txt', index_col=0)
nov_df = pd.read_csv('NOV2019_RIDECHECK_DATA.txt', index_col=0)
dec_df = pd.read_csv('DEC2019_RIDECHECK_DATA.txt', index_col=0)

/Users/shriyakaram/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
apc_df = jul_df.append(aug_df)
apc_df = apc_df.append(sep_df)
apc_df = apc_df.append(oct_df)
apc_df = apc_df.append(nov_df)
apc_df = apc_df.append(dec_df)
apc_df = apc_df.reset_index(drop=True)

In [13]:
apc_df.shape[0]

3816480

In [14]:
apc_df.dtypes

SCHEDULE_ID              int64
SCHEDULE_NAME           object
SIGNUP_NAME             object
SURVEY_DATE             object
SURVEY_STATUS            int64
SURVEY_TYPE              int64
SURVEY_SOURCE            int64
PATTERN_ID               int64
ROUTE_NUMBER           float64
ROUTE_NAME              object
DIRECTION_NAME          object
BRANCH                  object
SERVICE_CODE            object
SERVICE_TYPE            object
SERVICE_CLASS          float64
SERVICE_MODE            object
TRIP_START_TIME         object
TIME_PERIOD             object
SERVICE_PERIOD          object
TRIP_NUMBER              int64
TRIP_KEY                 int64
BLOCK_NUMBER             int64
BLOCK_KEY               object
BLOCK_ID                 int64
BLOCK_NAME             float64
RUN_NUMBER               int64
RUN_KEY                  int64
VEHICLE_NUMBER           int64
VEHICLE_DESCRIPTION     object
VEHICLE_SEATS          float64
REVENUE_START          float64
REVENUE_END            float64
REVENUE_

In [15]:
def fix_times (time) :
    """
    convert time to HH:MM format - removing 12/30/1899  in front of non-null times
    
    :param time: a time value 
    :return: time converted to HH:MM format
    """
    
    time = str(time)
    if time == 'NaN':
        return time

    # take last characters for hours:minutes:seconds format
    fixed_time = time[10:19]
    return fixed_time

In [20]:
for l in ['TRIP_START_TIME','TIME_SCHEDULED','TIME_ACTUAL_ARRIVE','TIME_ACTUAL_DEPART']:
    print(l)
    apc_df[l]= apc_df.swifter.set_npartitions(20).apply(lambda x: fix_times(x[l]),axis=1)

TRIP_START_TIME
TIME_SCHEDULED
TIME_ACTUAL_ARRIVE
TIME_ACTUAL_DEPART


In [21]:
apc_df.head(400)

,SCHEDULE_ID,SCHEDULE_NAME,SIGNUP_NAME,SURVEY_DATE,SURVEY_STATUS,SURVEY_TYPE,SURVEY_SOURCE,PATTERN_ID,ROUTE_NUMBER,ROUTE_NAME,DIRECTION_NAME,BRANCH,SERVICE_CODE,SERVICE_TYPE,SERVICE_CLASS,SERVICE_MODE,TRIP_START_TIME,TIME_PERIOD,SERVICE_PERIOD,TRIP_NUMBER,TRIP_KEY,BLOCK_NUMBER,BLOCK_KEY,BLOCK_ID,BLOCK_NAME,RUN_NUMBER,RUN_KEY,VEHICLE_NUMBER,VEHICLE_DESCRIPTION,VEHICLE_SEATS,REVENUE_START,REVENUE_END,REVENUE_NET,ODOM_START,ODOM_END,ODOM_NET,CONDITION_NUMBER,CHECKER_NAME,GARAGE_NAME,DIVISION_NAME,OPERATOR_ID,FAREBOX,MATCH_COUNT,COMMENTS,SORT_ORDER,STOP_ID,MAIN_CROSS_STREET,TRAVEL_DIRECTION,TIMEPOINT,SEGMENT_MILES,TIME_SCHEDULED,TIME_ACTUAL_ARRIVE,TIME_ACTUAL_DEPART,DWELL_TIME,RUNNING_TIME_ACTUAL,PASSENGERS_ON,PASSENGERS_OFF,PASSENGERS_IN,PASSENGERS_SPOT,WHEELCHAIRS,BICYCLES,MATCH_DISTANCE,TIMEPOINT_MILES,NON_STUDENT_FARE,CHILD,NR_BOARD,NR_ALIGHT,KNEELS,COMMENT_NUMBER,CHECKER_TIME,FIRST_LAST_STOP
0,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,10,354,SHOLAR/CARTA,S,0,0.85,,5:07:41,5:07:41,NaN,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1
1,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,20,100026,Sho CART,X,-1,NaN,5:06:00,5:07:34,5:08:02,NaN,8.51,0,0,0,NaN,0,0,335.0,1.96,NaN,NaN,NaN,NaN,0,NaN,NaN,1
2,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,30,505,RIVERSIDE/RIVER PARK,S,0,1.15,,5:10:58,5:10:58,0.00,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
3,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,40,784,MABEL/BATTERY PLACE,S,0,0.10,,5:12:40,5:12:40,0.00,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
4,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,50,283,3RD/HOUSTON,W,0,0.08,,5:13:13,5:13:13,0.00,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
5,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,60,284,3RD/LINDSAY,W,0,0.17,,5:13:23,5:13:23,0.00,NaN,0,0,0,NaN,0,0,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
6,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,70,285,3RD/HIGH,W,0,0.06,,5:13:51,5:13:51,0.00,NaN,0,0,0,NaN,0,0,1.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
7,109,May19 (Weekday),May 5 2019,7/1/2019 0:00:00,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN

In [22]:
#change 'outybound' to outbound (typo here)
apc_df.loc[apc_df['DIRECTION_NAME'] == 'OUTYBOUND', ['DIRECTION_NAME']] = 'OUTBOUND'

In [23]:
#cast dates to datetime object
apc_df['SURVEY_DATE'] = pd.to_datetime(apc_df['SURVEY_DATE'])

In [24]:
apc_df.dtypes

SCHEDULE_ID                     int64
SCHEDULE_NAME                  object
SIGNUP_NAME                    object
SURVEY_DATE            datetime64[ns]
SURVEY_STATUS                   int64
SURVEY_TYPE                     int64
SURVEY_SOURCE                   int64
PATTERN_ID                      int64
ROUTE_NUMBER                  float64
ROUTE_NAME                     object
DIRECTION_NAME                 object
BRANCH                         object
SERVICE_CODE                   object
SERVICE_TYPE                   object
SERVICE_CLASS                 float64
SERVICE_MODE                   object
TRIP_START_TIME                object
TIME_PERIOD                    object
SERVICE_PERIOD                 object
TRIP_NUMBER                     int64
TRIP_KEY                        int64
BLOCK_NUMBER                    int64
BLOCK_KEY                      object
BLOCK_ID                        int64
BLOCK_NAME                    float64
RUN_NUMBER                      int64
RUN_KEY     

In [25]:
apc_df.head()

,SCHEDULE_ID,SCHEDULE_NAME,SIGNUP_NAME,SURVEY_DATE,SURVEY_STATUS,SURVEY_TYPE,SURVEY_SOURCE,PATTERN_ID,ROUTE_NUMBER,ROUTE_NAME,DIRECTION_NAME,BRANCH,SERVICE_CODE,SERVICE_TYPE,SERVICE_CLASS,SERVICE_MODE,TRIP_START_TIME,TIME_PERIOD,SERVICE_PERIOD,TRIP_NUMBER,TRIP_KEY,BLOCK_NUMBER,BLOCK_KEY,BLOCK_ID,BLOCK_NAME,RUN_NUMBER,RUN_KEY,VEHICLE_NUMBER,VEHICLE_DESCRIPTION,VEHICLE_SEATS,REVENUE_START,REVENUE_END,REVENUE_NET,ODOM_START,ODOM_END,ODOM_NET,CONDITION_NUMBER,CHECKER_NAME,GARAGE_NAME,DIVISION_NAME,OPERATOR_ID,FAREBOX,MATCH_COUNT,COMMENTS,SORT_ORDER,STOP_ID,MAIN_CROSS_STREET,TRAVEL_DIRECTION,TIMEPOINT,SEGMENT_MILES,TIME_SCHEDULED,TIME_ACTUAL_ARRIVE,TIME_ACTUAL_DEPART,DWELL_TIME,RUNNING_TIME_ACTUAL,PASSENGERS_ON,PASSENGERS_OFF,PASSENGERS_IN,PASSENGERS_SPOT,WHEELCHAIRS,BICYCLES,MATCH_DISTANCE,TIMEPOINT_MILES,NON_STUDENT_FARE,CHILD,NR_BOARD,NR_ALIGHT,KNEELS,COMMENT_NUMBER,CHECKER_TIME,FIRST_LAST_STOP
0,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,10,354,SHOLAR/CARTA,S,0,0.85,,5:07:41,5:07:41,NaN,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1
1,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,20,100026,Sho CART,X,-1,NaN,5:06:00,5:07:34,5:08:02,NaN,8.51,0,0,0,NaN,0,0,335.0,1.96,NaN,NaN,NaN,NaN,0,NaN,NaN,1
2,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,30,505,RIVERSIDE/RIVER PARK,S,0,1.15,,5:10:58,5:10:58,0.0,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
3,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,40,784,MABEL/BATTERY PLACE,S,0,0.10,,5:12:40,5:12:40,0.0,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
4,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,50,283,3RD/HOUSTON,W,0,0.08,,5:13:13,5:13:13,0.0,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2


In [26]:
apc_df.to_csv('chattanooga_raw_apc_jul_through_dec_2019.csv')

# Drop APC duplicates
Need to look at duplicates on trip_key, survey_date, direction_name, stop_id, and sort_order. We can drop these (reduce to 1). The duplicates on trip_key, survey_date, direction_name, stop_id ONLY can be kept since these represent trips with loops. 

This code has been run on the Jul-Dec data and the cleaned dataset is located Teams under general > datasets > carta-occupancy > carta > Cleaned 2019 APC 

In [4]:
apc_df = pd.read_csv('chattanooga_raw_apc_jul_through_dec_2019.csv', index_col=0)

/Users/shriyakaram/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
apc_df.head()

,SCHEDULE_ID,SCHEDULE_NAME,SIGNUP_NAME,SURVEY_DATE,SURVEY_STATUS,SURVEY_TYPE,SURVEY_SOURCE,PATTERN_ID,ROUTE_NUMBER,ROUTE_NAME,DIRECTION_NAME,BRANCH,SERVICE_CODE,SERVICE_TYPE,SERVICE_CLASS,SERVICE_MODE,TRIP_START_TIME,TIME_PERIOD,SERVICE_PERIOD,TRIP_NUMBER,TRIP_KEY,BLOCK_NUMBER,BLOCK_KEY,BLOCK_ID,BLOCK_NAME,RUN_NUMBER,RUN_KEY,VEHICLE_NUMBER,VEHICLE_DESCRIPTION,VEHICLE_SEATS,REVENUE_START,REVENUE_END,REVENUE_NET,ODOM_START,ODOM_END,ODOM_NET,CONDITION_NUMBER,CHECKER_NAME,GARAGE_NAME,DIVISION_NAME,OPERATOR_ID,FAREBOX,MATCH_COUNT,COMMENTS,SORT_ORDER,STOP_ID,MAIN_CROSS_STREET,TRAVEL_DIRECTION,TIMEPOINT,SEGMENT_MILES,TIME_SCHEDULED,TIME_ACTUAL_ARRIVE,TIME_ACTUAL_DEPART,DWELL_TIME,RUNNING_TIME_ACTUAL,PASSENGERS_ON,PASSENGERS_OFF,PASSENGERS_IN,PASSENGERS_SPOT,WHEELCHAIRS,BICYCLES,MATCH_DISTANCE,TIMEPOINT_MILES,NON_STUDENT_FARE,CHILD,NR_BOARD,NR_ALIGHT,KNEELS,COMMENT_NUMBER,CHECKER_TIME,FIRST_LAST_STOP
0,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,10,354,SHOLAR/CARTA,S,0,0.85,NaN,5:07:41,5:07:41,NaN,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1
1,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,20,100026,Sho CART,X,-1,NaN,5:06:00,5:07:34,5:08:02,NaN,8.51,0,0,0,NaN,0,0,335.0,1.96,NaN,NaN,NaN,NaN,0,NaN,NaN,1
2,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,30,505,RIVERSIDE/RIVER PARK,S,0,1.15,NaN,5:10:58,5:10:58,0.0,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
3,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,40,784,MABEL/BATTERY PLACE,S,0,0.10,NaN,5:12:40,5:12:40,0.0,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2
4,109,May19 (Weekday),May 5 2019,2019-07-01,2,1,3,910,4.0,Route #4,OUTBOUND,[4]4PO,Route #4,NaN,NaN,Bus,5:06:00,AM Peak,Weekday,7,137030,7402,4044,0,NaN,15,15,752,BYD 2016 Electric,32.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,***Unknown Garage***,NaN,160551,NaN,114,Trip starts at 5:06a at Sho CART and ends at 6...,50,283,3RD/HOUSTON,W,0,0.08,NaN,5:13:13,5:13:13,0.0,NaN,0,0,0,NaN,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2


In [6]:
#drop all duplicates based on trip_key, survey_date, direction_name, stop_id, and sort_order
#later append this with subset of dups we want to keep
apc_no_dups = apc_df.drop_duplicates(['TRIP_KEY','SURVEY_DATE','DIRECTION_NAME','STOP_ID','SORT_ORDER'],keep=False)

In [7]:
#get all duplicates
apc_dups = apc_df.loc[apc_df.duplicated(subset=['TRIP_KEY','SURVEY_DATE','DIRECTION_NAME','STOP_ID','SORT_ORDER'], keep=False)]

In [8]:
apc_dups.shape[0]

5596

In [9]:
#drop routes 33, 34, 14
apc_dups = apc_dups[(apc_dups.ROUTE_NUMBER != 33) & (apc_dups.ROUTE_NUMBER != 34)
                                   & (apc_dups.ROUTE_NUMBER != 14)]

In [10]:
apc_dups.shape[0]

3874

In [11]:
#drop duplicates - reduce each duplicate pairing to 1
apc_dups = apc_dups.drop_duplicates(['TRIP_KEY','SURVEY_DATE','DIRECTION_NAME','STOP_ID','SORT_ORDER'],keep='first')

In [12]:
apc_dups.shape[0]

1937

In [19]:
#records that are duplicated on stop_id are from trip Ids 135974 and 138668 which are from route 10.9 that loops.
#The stops 292-293-294 are repeated in the trip but at a different stop sequence, so these are valid duplicates.
apc_stop_dups = apc_no_dups.loc[apc_no_dups.duplicated(subset=['TRIP_KEY','SURVEY_DATE','DIRECTION_NAME','STOP_ID'], keep=False)]

In [23]:
#append 1)APC without dups WITH 2)APC with dups we want to keep
apc_cleaned = apc_no_dups.append(apc_dups)

In [25]:
apc_cleaned.shape[0]

3812821

In [24]:
apc_no_dups.shape[0] + apc_dups.shape[0]

3812821

In [26]:
#sort cleaned data
cleaned_chattanooga_apc_jul19_through_dec19 = apc_cleaned.sort_values(['TRIP_KEY', 'SURVEY_DATE', 'STOP_ID', 'SORT_ORDER'], ascending=False)

In [27]:
cleaned_chattanooga_apc_jul19_through_dec19.to_csv('cleaned_chattanooga_apc_jul19_through_dec19.csv')

# Merge with 2019 GTFS

I am still waiting on the 2019 GTFS. The code to run should be the same as the 2020 data. The merge code for Carta is located in the Github here https://github.com/smarttransit-ai/transit-occupancy-dashboard/blob/master/app/dataprep/chattanooga/CARTA_data_processing_updated.ipynb.

Follow the steps in the processing notebook above starting at the section 'Start the GTFS Join - load GTFS data' and run the code until 'Join RideCheck Stop Data'. Note: the only difference would be the part about dealing with the extra 204 entries from the join, because we have to look at trips 135974 AND 138668. To fix this issue: run the section 'Getting extra 204 during join' for both trip IDs (with the 2020 data, we only had to deal with trip ID 138668). 

In [4]:
apc_df = pd.read_csv('cleaned_chattanooga_apc_jul19_through_dec19.csv', index_col=0)

/Users/shriyakaram/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
